# Capstone_Project_with_Immigration_Data
### Data Engineering Capstone Project

#### Project Summary (describe the project at a high level)

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# import all libraries and install the packages 
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>   ( choose some tasks after knowing the datasets better) 
* I want to see which coutry sends the most visitors to US ? 
* which visa type is the most popular ? 
* The most common airlines during tourism seasons ?    
* what is the most popular season to visit US ? 
* which cities in US attract the most visitors ? 

#### Tools :
Redshift , S3 , Spark, Postgres

#### Describe and Gather Data 
* I94 immigration data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. There are 28 columns for each row. The following are some relavant attributes:
  * i94yr = 4 digit year
  * i94mon = numeric month
  * i94cit = 3 digit code of origin city
  * i94port = 3 character code of destination USA city
  * arrdate = arrival date in the USA
  * i94mode = 1 digit travel code
  * depdate = departure date from the USA
  * i94visa = reason for immigration
* data source:  https://travel.trade.gov/research/reports/i94/historical/2016.html

  
* U.S. City Demographic Data: This dataset contains information about the demographics of all US cities and census-designated places with a   
  population greater or equal to 65,000. This data comes from the US Census Bureau's 2015 American Community Survey. It comes with 12  
  columns.
  * data source: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/
* Airport Code Table: This is a simple table of airport codes and corresponding cities. 
  * data source: https://datahub.io/core/airport-codes#data

#### Explore Sample data for each dataset 

##### (1). I94 dataset

In [9]:
# Read in the data here
# take a look at the sample data in csv format before reading it all in
fname_i94 = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_i94 = pd.read_sas(fname_i94, 'sas7bdat', encoding="ISO-8859-1")

In [4]:
df_i94.head()

cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  i94mode i94addr  \
0    6.0  2016.0     4.0   692.0   692.0     XXX  20573.0      NaN     NaN   
1    7.0  2016.0     4.0   254.0   276.0     ATL  20551.0      1.0      AL   
2   15.0  2016.0     4.0   101.0   101.0     WAS  20545.0      1.0      MI   
3   16.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   
4   17.0  2016.0     4.0   101.0   101.0     NYC  20545.0      1.0      MA   

   depdate   ...     entdepu  matflag  biryear   dtaddto gender insnum  \
0      NaN   ...           U      NaN   1979.0  10282016    NaN    NaN   
1      NaN   ...           Y      NaN   1991.0       D/S      M    NaN   
2  20691.0   ...         NaN        M   1961.0  09302016      M    NaN   
3  20567.0   ...         NaN        M   1988.0  09302016    NaN    NaN   
4  20567.0   ...         NaN        M   2012.0  09302016    NaN    NaN   

  airline        admnum  fltno visatype  
0     NaN  1.897628e+09    NaN       B2  
1     NaN  3.736796e+09  00296       F1  
2      OS  6.666432e+08     93       B2  
3      AA  9.246846e+10  00199       B2  
4      AA  9.246846e+10  00199       B2  

[5 rows x 28 columns]

##### (2). Land temperature data

In [5]:
fname_temp = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(fname_temp)

In [6]:
df_temp.head()

dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E

##### (3). Cities demographics data

In [20]:
fname_city = 'us-cities-demographics.csv'
df_city = pd.read_csv(fname_city, delimiter = ";")    # origianl file is semi colon separated 

In [21]:
df_city.head()

City          State  Median Age  Male Population  \
0     Silver Spring       Maryland        33.8          40601.0   
1            Quincy  Massachusetts        41.0          44129.0   
2            Hoover        Alabama        38.5          38040.0   
3  Rancho Cucamonga     California        34.5          88127.0   
4            Newark     New Jersey        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         Hispanic or Latino  25924  
1                    2.39         MA                      White  58723  
2                    2.58         AL                      Asian   4759  
3                    3.18         CA  Black or African-American  24437  
4                    2.73         NJ                      White  76402

##### (4). Aiport code data

In [23]:
fname_airport = 'airport-codes.csv'
df_airport = pd.read_csv(fname_airport)

In [24]:
df_airport.head()

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [8]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 2: Explore and Assess the Data

#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [4]:
# Data cleaning


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
​
#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:

Integrity constraints on the relational database (e.g., unique key, data type, etc.)
Unit tests for the scripts to ensure they are doing the right thing
Source/Count checks to ensure completeness
Run Quality Checks

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

### Reference:
##### Part 1:
##### https://github.com/cheuklau/udacity-capstone
##### https://github.com/shalgrim/dend_capstone


##### Part 2:
##### https://github.com/FedericoSerini/DEND-Capstone-RomeTransportNetwork
##### https://github.com/sariabod/dend-capstone
##### https://github.com/akessela/Sparkify-Capstone-Project
##### https://github.com/Mikemraz/Capstone-Project-Big-Data-Sparkify